In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# SymPy helper functions

The {mod}`ampform.sympy` module contains a few classes that make it easier to construct larger expressions that consist of several mathematical definitions.

## Unevaluated expressions

The {func}`.unevaluated` decorator makes it easier to write classes that represent a mathematical function definition. It makes a class that derives from {class}`sp.Expr <sympy.core.expr.Expr>` behave more like a {func}`~.dataclasses.dataclass` (see [PEP&nbsp;861](https://peps.python.org/pep-0681)). All you have to do is:

1. Specify the arguments the function requires.
2. Specify how to render the 'unevaluated' or 'folded' form of the expression with a `_latex_repr_` string or method.
3. Specify how to unfold the expression using an `evaluate()` method.

In the example below, we define a phase space factor $\rho^\text{CM}$ using the Chew-Mandelstam function (see PDG Resonances section, [Eq.&nbsp;(50.44)](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=15)). For this, you need to define a break-up momentum $q$ as well.

In [ ]:
import sympy as sp

from ampform.sympy import unevaluated


@unevaluated(real=False)
class PhspFactorSWave(sp.Expr):
    s: sp.Symbol
    m1: sp.Symbol
    m2: sp.Symbol
    _latex_repr_ = R"\rho^\text{{CM}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        cm = (
            (2 * q / sp.sqrt(s))
            * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
            - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
        ) / (16 * sp.pi**2)
        return 16 * sp.pi * sp.I * cm


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: sp.Symbol
    m1: sp.Symbol
    m2: sp.Symbol
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - (m1 + m2) ** 2) * (s - (m1 - m2) ** 2) / (s * 4))

As can be seen, the LaTeX rendering of these classes makes them ideal for mathematically defining and building up larger amplitude models:

In [ ]:
from IPython.display import Math

from ampform.io import aslatex

s, m1, m2 = sp.symbols("s m1 m2")
q_expr = BreakupMomentum(s, m1, m2)
rho_expr = PhspFactorSWave(s, m1, m2)
Math(aslatex({e: e.evaluate() for e in [rho_expr, q_expr]}))

Class variables and default arguments to instance arguments are also supported. They can either be indicated with {class}`typing.ClassVar` or by not providing a type hint:

In [ ]:
from __future__ import annotations

from typing import Any, ClassVar


@unevaluated
class FunkyPower(sp.Expr):
    x: Any
    m: int = 1
    default_return: ClassVar[sp.Expr | None] = None
    class_name = "my name"
    _latex_repr_ = R"f_{{{m}}}\left({x}\right)"

    def evaluate(self) -> sp.Expr | None:
        if self.default_return is None:
            return self.x**self.m
        return self.default_return


x = sp.Symbol("x")
exprs = (
    FunkyPower(x),
    FunkyPower(x, 2),
    FunkyPower(x, m=3),
)
Math(aslatex({e: e.doit() for e in exprs}))

In [ ]:
FunkyPower.default_return = sp.Rational(0.5)
Math(aslatex({e: e.doit() for e in exprs}))

## Integral

Integral over $x^p$ with symbolic range from a to b


In [ ]:
import sympy as sp

from ampform.sympy import UnevaluatableIntegral

x, p, a, b = sp.symbols("x,p,a,b")
integral_expr = UnevaluatableIntegral(x**p, (x, a, b))
integral_expr.doit()

In [ ]:
integral_func = sp.lambdify(args=[p, a, b], expr=integral_expr)

In [ ]:
import numpy as np

value_a = 1.2
value_b = 3.6
value_p = np.array([0.4, 0.6, 0.8])

areas = integral_func(value_p, value_a, value_b)
areas

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import matplotlib.pyplot as plt

x_area = np.linspace(value_a, value_b, num=100)
x_line = np.linspace(0, 4, num=100)

fig, ax = plt.subplots()
ax.set_xlabel("$x$")
ax.set_ylabel("$x^p$")
ax.plot(x_line, x_line ** value_p[0], label="$p=0.4$", color="C0")
ax.fill_between(x_area, x_area ** value_p[0], alpha=1, color="C0")

ax.plot(x_line, x_line ** value_p[1], label="$p=0.6$", color="C1")
ax.fill_between(x_area, x_area ** value_p[1], alpha=0.6, color="C1")

ax.plot(x_line, x_line ** value_p[2], label="$p=0.8$", color="C2")
ax.fill_between(x_area, x_area ** value_p[2], alpha=0.7, color="C2")

ax.text(
    (value_a + value_b) / 2,
    ((value_a ** value_p[0] + value_b ** value_p[0]) / 2) * 0.5,
    "Area",
    horizontalalignment="center",
    verticalalignment="center",
)
ax.annotate(
    "a", (value_a, 0.08), textcoords="offset points", xytext=(0, -15), ha="center"
)
ax.annotate(
    "b", (value_b, 0.08), textcoords="offset points", xytext=(0, -15), ha="center"
)

ax.legend()
plt.show()

There are numerous functions whose integrals cannot be expressed in terms of known special functions and for which no closed-form solutions have been identified yet. Here are a few examples:

1. **Integral Involving Complex Exponential and Polynomials:**
   $$ \int e^{x + e^{-x}} dx $$
   This integral combines an exponential function with a nested exponential term, leading to a complexity that resists simplification in terms of known functions.

2. **Integral of a Rational Function Times an Exponential:**
   $$ \int \frac{e^x}{x^2 + 1} dx $$
   This integral involves the exponential function and a non-trivial rational function, which together do not yield to standard integration techniques.

3. **Integral of an Exponential of a Trigonometric Function:**
   $$ \int e^{\sin x} dx $$
   The combination of an exponential function and a sinusoidal function is another case where known methods of integration in terms of elementary or special functions fail.

4. **Product of a Polynomial and a Trigonometric Function:**
   $$ \int x^4 \sin(x^2) dx $$
   This integral combines a polynomial with a trigonometric function in a non-standard way, making it resistant to analytical methods.

5. **Integral of a Logarithmic Function Inside an Exponential:**
   $$ \int e^{\ln(x)/x} dx $$
   The combination of a logarithmic function inside an exponential function, as seen here, creates a complex relationship not easily resolved by known functions.

In the context of hadron physics and high-energy physics, encountering such challenging integrals is not uncommon. They can arise in theoretical models, complex scattering problems, or in the analysis of experimental data. In such cases, physicists often resort to numerical integration, computational approximations, or the development of new special functions or methods tailored to the problem at hand. These integrals, while not solvable in a traditional sense, can still provide significant insights into the behaviors and properties of the systems being studied.

Some example test to compare ``sp.Integral`` and ``UnevaluatableIntegral``

In [ ]:
UnevaluatableIntegral(sp.exp(-(x**4)), (x, a, b)).doit()

In [ ]:
sp.Integral(sp.exp(-(x**4)), (x, a, b)).doit()

In [ ]:
UnevaluatableIntegral(sp.sin(x) / x, (x, a, b)).doit()

In [ ]:
sp.Integral(sp.sin(x) / x, (x, a, b)).doit()

In [ ]:
UnevaluatableIntegral(sp.exp(x + sp.exp(-x)), (x, a, b)).doit()

In [ ]:
sp.Integral(sp.exp(x + sp.exp(-x)), (x, a, b)).doit()

In [ ]:
UnevaluatableIntegral(sp.exp(sp.log(x) / x), (x, a, b)).doit()

In [ ]:
sp.Integral(sp.exp(sp.log(x) / x), (x, a, b)).doit()

In [ ]:
UnevaluatableIntegral(sp.exp(sp.sin(x)), (x, a, b)).doit()

In [ ]:
sp.Integral(sp.exp(sp.sin(x)), (x, a, b)).doit()

In [ ]:
UnevaluatableIntegral(sp.exp(x) / (x**2 + 1), (x, a, b)).doit()

In [ ]:
sp.Integral(sp.exp(x) / (x**2 + 1), (x, a, b)).doit()

## Summations

The {class}`.PoolSum` class makes it possible to write sums over non-integer ranges. This is for instance useful when summing over allowed helicities. Here are some examples:

In [ ]:
from ampform.sympy import PoolSum

i, j, m, n = sp.symbols("i j m n")
expr = PoolSum(i**m + j**n, (i, (-1, 0, +1)), (j, (2, 4, 5)))
Math(aslatex({expr: expr.doit()}))

In [ ]:
import numpy as np

A = sp.IndexedBase("A")
λ, μ = sp.symbols("lambda mu")
to_range = lambda a, b: tuple(sp.Rational(i) for i in np.arange(a, b + 0.5))
expr = abs(PoolSum(A[λ, μ], (λ, to_range(-0.5, +0.5)), (μ, to_range(-1, +1)))) ** 2
Math(aslatex({expr: expr.doit()}))